In [1]:
import pandas as pd
import numpy as np

# load the dataset
df = pd.read_csv('cancer.csv')

# display the first few rows of the dataset
df.head(2)

,"diagnosis(1=m, 0=b)",radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [2]:
df = df.rename(columns = {'diagnosis(1=m, 0=b)': 'condition'})

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(['condition'], axis=1), 
                                                    df['condition'], 
                                                    test_size=0.2, 
                                                    random_state=42)


### With out Using Lambda Function :

In [4]:
import time

start = time.time()


class NaiveBayesClassifier1:
    def __init__(self, alpha=1):
        self.alpha = alpha  # Laplace smoothing parameter
        self.classes = None  # list of class labels
        self.class_priors = None  # dictionary of class priors
        self.class_likelihoods = None  # dictionary of class conditional likelihoods
    
    def fit(self, X, y):
        self.classes = list(set(y))
        self.class_priors = {}
        self.class_likelihoods = {}
        
        # calculate class priors
        for c in self.classes:
            self.class_priors[c] = (y == c).sum() / len(y)
        
        # calculate class conditional likelihoods
        for c in self.classes:
            # get the subset of training data belonging to class c
            X_c = X[y == c]
            # calculate the mean and variance of each feature in X_c
            means = X_c.mean(axis=0)
            variances = X_c.var(axis=0) + self.alpha
            # store the mean and variance for class c
            self.class_likelihoods[c] = (means, variances)
    
    def predict(self, X):
        # calculate the log posterior probability of each class for each instance in X
        log_posteriors = []
        for x in X:
            log_p_c = {}
            for c in self.classes:
                # calculate the log prior probability of class c
                log_p_c[c] = np.log(self.class_priors[c])
                # calculate the log likelihood of x given class c
                means, variances = self.class_likelihoods[c]
                log_likelihood = np.sum(-0.5 * np.log(2 * np.pi * variances) - 0.5 * ((x - means) ** 2 / variances), axis=0)
                log_p_c[c] += log_likelihood.sum()
            log_posteriors.append(log_p_c)
        
        # predict the class with the highest log posterior probability for each instance in X
        y_pred = []
        for log_p_c in log_posteriors:
            max_log_p = -np.inf
            max_c = None
            for c, log_p in log_p_c.items():
                if log_p > max_log_p:
                    max_log_p = log_p
                    max_c = c
            y_pred.append(max_c)
        return y_pred
    
#create an instance of the NaiveBayesClassifier class
nb1 = NaiveBayesClassifier1()

#fit the training data
nb1.fit(X_train.values, y_train.values)

#make predictions on the testing data
y_pred = nb1.predict(X_test.values)

#calculate accuracy
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.2f}')


end = time.time()

print('\n')
print("Time taken to run code in Build in code in seconds -- ",end - start)

print('\n')
print("Time taken to run code in build in code in milli seconds -- ",((end - start)*1000))

Accuracy: 0.96


Time taken to run code in Build in code in seconds --  0.008452653884887695


Time taken to run code in build in code in milli seconds --  8.452653884887695


In [5]:

import time

start = time.time()


#create an instance of the NaiveBayesClassifier class
nb1 = NaiveBayesClassifier1()

#fit the training data
nb1.fit(X_train.values, y_train.values)

end = time.time()

print('\n')
print("Time taken to run code in build code in seconds -- ",(end - start)*1000)




Time taken to run code in build code in seconds --  2.003908157348633


### Using Lambda Function :

In [6]:

import time

start = time.time()

class NaiveBayesClassifier:
    def __init__(self, alpha=1):
        self.alpha = alpha  # Laplace smoothing parameter
        self.classes = None  # list of class labels
        self.class_priors = None  # dictionary of class priors
        self.class_likelihoods = None  # dictionary of class conditional likelihoods
    
    def fit(self, X, y):
        self.classes = list(set(y))
        self.class_priors = {}
        self.class_likelihoods = {}
        
        # calculate class priors
        for c in self.classes:
            self.class_priors[c] = (y == c).sum() / len(y)
        
        # calculate class conditional likelihoods
        for c in self.classes:
            # get the subset of training data belonging to class c
            X_c = X[y == c]
            # calculate the mean and variance of each feature in X_c
            means = X_c.mean(axis=0)
            variances = X_c.var(axis=0) + self.alpha
            # store the mean and variance for class c
            self.class_likelihoods[c] = (means, variances)
    
    def predict(self, X):
        # calculate the log posterior probability of each class for each instance in X
        log_posteriors = list(map(lambda x: {c: np.log(self.class_priors[c]) +
                                np.sum(-0.5 * np.log(2 * np.pi * self.class_likelihoods[c][1]) -
                                       0.5 * ((x - self.class_likelihoods[c][0]) ** 2 / self.class_likelihoods[c][1]).sum())
                                for c in self.classes}, X))
        
        # predict the class with the highest log posterior probability for each instance in X
        y_pred = list(map(lambda p: max(p, key=p.get), log_posteriors))
        return y_pred

    #create an instance of the NaiveBayesClassifier class
nb = NaiveBayesClassifier()

#fit the training data
nb.fit(X_train.values, y_train.values)

#make predictions on the testing data
y_pred = nb.predict(X_test.values)

#calculate accuracy
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.2f}')


end = time.time()

print('\n')
print("Time taken to run code in build code in seconds -- ",end - start)

print('\n')
print("Time taken to run code in Sklearn in milli seconds -- ",((end - start)*1000))


Accuracy: 0.92


Time taken to run code in build code in seconds --  0.007998943328857422


Time taken to run code in Sklearn in milli seconds --  7.998943328857422


In [7]:
import time

start = time.time()


#create an instance of the NaiveBayesClassifier class
nb = NaiveBayesClassifier1()

#fit the training data
nb.fit(X_train.values, y_train.values)

end = time.time()

print('\n')
print("Time taken to run code in build code in seconds -- ",(end - start)*1000)




Time taken to run code in build code in seconds --  4.004955291748047


### Using Sklearn :

In [8]:
import time

start = time.time()

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)



from sklearn.metrics import accuracy_score, precision_score, recall_score
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy: {:.2f}, Precision: {:.2f}, Recall: {:.2f}".format(accuracy, precision, recall))


end = time.time()

print('\n')
print("Time taken to run code in Sklearn in seconds -- ",end - start)

print('\n')
print("Time taken to run code in Sklearn in milli seconds -- ",((end - start)*1000))

Accuracy: 0.97, Precision: 1.00, Recall: 0.93


Time taken to run code in Sklearn in seconds --  0.016437530517578125


Time taken to run code in Sklearn in milli seconds --  16.437530517578125


In [9]:
import time

start = time.time()

clf.fit(X_train, y_train)

end = time.time()

print('\n')
print("Time taken to run code in Sklearn in seconds -- ",(end - start)*1000)




Time taken to run code in Sklearn in seconds --  2.996683120727539


### Final Report : 

In [10]:
import pandas as pd

# code 2
result1 = [2.0039, "96 %"]
df1 = pd.DataFrame({'without using lambda ': result1})

# code 3
result2 = [4.00495, "92 %"]
df2 = pd.DataFrame({'using lambda': result2})

# code 1
result4 = [2.9966, "97 %"]
df4 = pd.DataFrame({'using sklearn': result4})

# code 4
result3 = ['Time stamp', 'accuracy']
df3 = pd.DataFrame({'Column': result3})


# display the results in a table
df = pd.concat([df3, df1, df2, df4], axis=1)

print("\n")
print(df)




       Column without using lambda  using lambda using sklearn
0  Time stamp                2.0039      4.00495        2.9966
1    accuracy                  96 %         92 %          97 %


#### * Thank you